In [9]:
!pip install yfinance
!pip install yahooquery
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model


In [57]:
clmns = [
    "Open 12-11","Open 11-10","Open 10-9","Open 9-8","Open 8-7","Open 7-6","Open 6-5","Open 5-4","Open 4-3","Open 3-2","Open 2-1",
    "Close 12-11","Close 11-10","Close 10-9","Close 9-8","Close 8-7","Close 7-6","Close 6-5","Close 5-4","Close 4-3","Close 3-2","Close 2-1",
    "High 12-11","High 11-10","High 10-9","High 9-8","High 8-7","High 7-6","High 6-5","High 5-4","High 4-3","High 3-2","High 2-1",
    "Low 12-11","Low 11-10","Low 10-9","Low 9-8","Low 8-7","Low 7-6","Low 6-5","Low 5-4","Low 4-3","Low 3-2","Low 2-1",
    "AdjV 12-11","AdjV 11-10","AdjV 10-9","AdjV 9-8","AdjV 8-7","AdjV 7-6","AdjV 6-5","AdjV 5-4","AdjV 4-3","AdjV 3-2","AdjV 2-1",
    "Volume 12-11","Volume 11-10","Volume 10-9","Volume 9-8","Volume 8-7","Volume 7-6","Volume 6-5","Volume 5-4","Volume 4-3","Volume 3-2","Volume 2-1",
    "suggestion"
]

def read_syms_from_txt():
  with open("syms.txt", "r") as f:
    lines = f.readlines()
    f.close()
  lst = []
  for line in lines:
    line = line.strip()
    lst.append(line)
  symbols = lst
  return symbols
def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["yes","no"], axis=1)
  y = data[["yes","no"]]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.1)
  print(xTrain.shape, end=" ")
  print(yTrain.shape)
  print(xTest.shape, end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def scaler(row):
    scaler = MinMaxScaler(feature_range=(-5, 5))
    row = scaler.fit_transform(row)
    return row
def process(data):
    data = data.dropna()
    row = []
    data = np.array(data)
    llst = [0, 1, 2, 3, 4, 5]

    for i in range(12, data.shape[0]):
        grow = []
        srow = []

        for j in llst:

           srow.append([
               data[i-1][j] - data[i-2][j],
               data[i-2][j] - data[i-3][j],
               data[i-3][j] - data[i-4][j],
               data[i-4][j] - data[i-5][j],
               data[i-5][j] - data[i-6][j],
               data[i-6][j] - data[i-7][j],
               data[i-7][j] - data[i-8][j],
               data[i-8][j] - data[i-9][j],
               data[i-9][j] - data[i-10][j],
               data[i-10][j] - data[i-11][j],
               data[i-11][j] - data[i-12][j]
           ])

        for lst in srow:
            mm = np.array(lst)
            mm = np.reshape(mm, (-1, 1))
            grow.append(scaler(mm))

        sugg = "no"
        if data[i][4] > data[i][0]:
            sugg = "yes"

        arr = np.array(grow).flatten()
        arr = np.append(arr, sugg)
        row.append(arr)


    grow = []
    srow = []
    llst = []
    data = []
    arr = []
    mm = []

    return np.array(row)

In [71]:
def get_data(symbols):
    unattached_dfs = []
    for symbol in symbols[:5]:
         data = yf.download(symbol, period="MAX",
                            interval="1d", progress=False)
         if data.empty:
              pass
         else:
               if data.shape[0] > 12:
                   unattached_dfs.append(
                       process(data))

    data = np.array(unattached_dfs[0])
    for i in unattached_dfs[1:]:
        data = np.append(data, np.array(i), axis=0)
        
    symbol = []

    data = pd.DataFrame(data, columns=clmns)
    sugg = data["suggestion"]
    data.drop("suggestion",axis=1,inplace=True)
    sugg = pd.get_dummies(sugg)
    data = pd.concat([data,sugg],axis=1)
    data = data.astype(float)

    xTrain, xTest, yTrain, yTest = spliting(data)
    print(data.shape)
    data.to_csv(f"{random.randint(1,10000)}.csv")
    #data = []
    return xTrain, xTest, yTrain, yTest,data



symbols = read_syms_from_txt()
#symbols = get_crypto_syms()

#pieces = 15
#new_arrays = np.array_split(symbols, pieces)
#for symbols in new_arrays[7:]:
#    get_data(symbols)

xTrain, xTest, yTrain, yTest,data= get_data(symbols)
data 


1 Failed download:
- PIH: No data found, symbol may be delisted

1 Failed download:
- PIHPP: No data found, symbol may be delisted
(10252, 66) (10252, 2)
(1140, 66) (1140, 2)
(11392, 68)


,Open 12-11,Open 11-10,Open 10-9,Open 9-8,Open 8-7,Open 7-6,Open 6-5,Open 5-4,Open 4-3,Open 3-2,...,Volume 9-8,Volume 8-7,Volume 7-6,Volume 6-5,Volume 5-4,Volume 4-3,Volume 3-2,Volume 2-1,no,yes
0,-1.087999,-1.112001,-5.000000,0.855998,-0.959999,-0.648002,5.000000,0.743998,3.239997,-0.008000,...,4.437149,3.296354,3.600420,2.048258,5.000000,4.102759,2.609599,-5.000000,1.0,0.0
1,0.167998,-1.087999,-1.112001,-5.000000,0.855998,-0.959999,-0.648002,5.000000,0.743998,3.239997,...,-1.691149,3.093157,-0.771662,0.258460,-5.000000,5.000000,1.960301,-3.098271,1.0,0.0
2,-0.200001,0.167998,-1.087999,-1.112001,-5.000000,0.855998,-0.959999,-0.648002,5.000000,0.743998,...,1.970528,-1.691149,3.093157,-0.771662,0.258460,-5.000000,5.000000,1.960301,1.0,0.0
3,-2.752000,-0.200001,0.167998,-1.087999,-1.112001,-5.000000,0.855998,-0.959999,-0.648002,5.000000,...,0.529007,1.970528,-1.691149,3.093157,-0.771662,0.258460,-5.000000,5.000000,0.0,1.0
4,0.224000,-2.752000,-0.200001,0.167998,-1.087999,-1.112001,-5.000000,0.855998,-0.959999,-0.648002,...,-0.910396,1.831706,3.612866,-0.911545,5.000000,0.224584,1.497415,-5.000000,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11387,-0.555567,-1.111108,-1.111135,3.888865,-5.000000,-3.333351,5.000000,0.555541,-4.444459,-0.000026,...,0.966387,-5.000000,4.201681,2.142857,4.873950,0.966387,5.000000,0.210084,1.0,0.0
11388,-0.555567,-0.555567,-1.111108,-1.111135,3.888865,-5.000000,-3.333351,5.000000,0.555541,-4.444459,...,2.815126,0.966387,-5.000000,4.201681,2.142857,4.873950,0.966387,5.000000,0.0,1.0
11389,-2.777784,-0.555567,-0.555567,-1.111108,-1.111135,3.888865,-5.000000,-3.333351,5.000000,0.555541,...,-0.276596,2.914894,1.042553,-5.000000,4.319149,2.234043,5.000000,1.042553,1.0,0.0
11390,1.666649,-2.777784,-0.555567,-0.555567,-1.111108,-1.111135,3.888865,-5.000000,-3.333351,5.000000,...,2.918367,-0.469388,2.591837,0.795918,-5.000000,3.938776,1.938776,4.591837,1.0,0.0


In [ ]:
model = Sequential()



model.add(Dense(128, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(32, activation='relu'))


model.add(Dense(2, activation='sigmoid'))
model.compile(optimizer="adam", loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_66 (Dense)            (None, 128)               8576      
                                                                 
 dense_67 (Dense)            (None, 128)               16512     
                                                                 
 dense_68 (Dense)            (None, 128)               16512     
                                                                 
 dense_69 (Dense)            (None, 32)                4128      
                                                                 


In [ ]:
model.fit(xTrain,yTrain,epochs=35,batch_size=256,validation_data=(xTest,yTest))

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
model = load_model("/content/drive/MyDrive/[0.23090815544128418, 0.9449216723442078].h5")

In [ ]:
model.fit(xTrain,yTrain,epochs=10,batch_size=55555,validation_data=(xTest,yTest))